In [1]:
import ctypes
import sys, os
sys.path.append("../..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pyoptix
from pyoptix.classes import (Beamline, GaussianSource, RecordingMode, 
                             ConicCylindricalMirror, PlaneFilm)


intialzing SR library
OptiX library initialized


In [2]:
hc = h*c/eV
pyoptix.output_notebook()

Loading BokehJS ...

In [3]:
Bl = Beamline(name="KB")
source = GaussianSource(name="gaussSource")
HFM = ConicCylindricalMirror(name="HFM")
VFM = ConicCylindricalMirror(name="VFM")
plan_foc = PlaneFilm(name="foc")

In [4]:
source.nrays = 5000
source.sigma_x = 100e-6 #m
source.sigma_y = 100e-6 #m
source.sigma_x_div = 1e-3 # rad
source.sigma_y_div = 1e-3 # rad

In [5]:
p = 1.8 #m
q = 0.05 #m
theta = 10*degree
dist = 0.04 #m

In [6]:
HFM.distance_from_previous = p
HFM.phi = -90*degree # rad
HFM.theta = theta # rad
HFM.inverse_p = -1/p # m-1
HFM.inverse_q = 1/(q + dist) # m-1
HFM.theta0 = theta #rad
HFM.next = VFM

In [7]:
VFM.distance_from_previous = dist
VFM.phi = -90*degree # rad
VFM.theta = theta # rad
VFM.inverse_p = -1/(p + dist) # m-1
VFM.inverse_q = 1/(q) # m-1
VFM.theta0 = theta #rad
VFM.next = plan_foc

In [8]:
plan_foc.recording_mode = RecordingMode.recording_output
plan_foc.distance_from_previous = q

In [9]:
HFM.dump_properties()
print(HFM._element_type)
VFM.dump_properties()
plan_foc.dump_properties()

Dump of all optix parameter for HFM
	 DX: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 DY: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 DZ: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 Dphi: 0.0 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 Dpsi: 0.0 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 Dtheta: 0.0 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 distance: 1.8 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 invp: -0.5555555555555556 [0.0, 0.0],x1.0, type -1, groupe 1, flags 0
	 invq: 11.11111111111111 [0.0, 0.0],x1.0, type -1, groupe 1, flags 0
	 phi: -1.5707963267948966 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 psi: 0.0 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 theta: 0.17453292519943295 [0.0, 0.0],x1.0, type 1, groupe 0, flags 0
	 theta0: 0.17453292519943295 [0.0, 0.0],x1.0, type 1, groupe 1, flags 0
ConicBaseCylindricalMirror
Dump of all optix parameter for VFM
	 DX: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0
	 DY: 0.0 [0.0, 0.0],x1.0, type 2, groupe 0, flags 0

In [10]:
Bl.chains["chain"] = [source, HFM, VFM, plan_foc]
Bl.active_chain = "chain"
Bl.clear_impacts(clear_source=True)
Bl.align(20e-9)

Bl.generate(20e-9)

Bl.radiate()

Chaîne chain:
	gaussSource -> HFM -> VFM -> foc 


1

In [11]:
Bl.draw_active_chain()

gaussSource -> HFM -> VFM -> foc 


In [12]:
plan_foc.show_diagram(distance_from_oe=0)

test chain


({'xy': ColumnDataSource(id='1189', ...),
  'xxp': ColumnDataSource(id='1190', ...),
  'yyp': ColumnDataSource(id='1191', ...)},
 [Column(id='1349', ...), Column(id='1507', ...), Column(id='1665', ...)])

In [13]:
diag = plan_foc.get_diagram()
print(diag["X"].std())
print(diag["Y"].std())
print(diag["dX"].std())
print(diag["dY"].std())

5.003750807717317e-06
2.796104910598094e-06
0.02012700705421226
0.039599568473388365
